In [1]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import mne
import pywt
from utils import*
from tqdm import tqdm
import gc 
from random import randint
# plt.rcParams['figure.dpi'] = 100
# plt.rcParams['figure.figsize'] = [224/100,224/100]
import time
import multiprocessing
import pandas as pd

In [2]:
scales = np.arange(1,24)        # Scales for CWT
csvdir = '/home/dll-1/Desktop/eeg/datasets/Adil paper/csv/SW & SSW CSV Files'     # Directory of CSV Files
edfdir = '/home/dll-1/Desktop/eeg/datasets/Adil paper/edf/Abnormal EDF Files'	  # Directory of Abnormal EDF files
main_path = '/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest/' 
normalEdf_dir = '/home/dll-1/Desktop/eeg/datasets/Adil paper/edf/Normal EDF Files/'
data_csv = '/home/dll-1/Desktop/eeg/datasets/Adil paper/data_split.csv'

In [3]:
Label_data = np.empty((0,19))
Epochs_data = np.empty((0,19,400))

dest_list = ['Normal', 'Slowing Waves', 'Spike and Sharp waves']  # Names of subfolders with in main folder
waveletsTypes = ['mexh','morl', 'gaus1', 'gaus2']
splits = ['train', 'valid', 'test']

allFileList = os.listdir(csvdir)
allEdfList = os.listdir(normalEdf_dir)


df = pd.read_csv(data_csv)


train_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'train')]
train_abnormal = train_abnormal['File'].tolist()

test_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'test')]
test_abnormal = test_abnormal['File'].tolist()

valid_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'valid')]
valid_abnormal = valid_abnormal['File'].tolist()


train_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'train')]
train_normal = train_normal['File'].tolist()

test_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'test')]
test_normal = test_normal['File'].tolist()

valid_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'valid')]
valid_normal = valid_normal['File'].tolist()

In [4]:
for waveletType in waveletsTypes:
    print(main_path + waveletType)
    if not os.path.exists(main_path + waveletType):
        os.makedirs(main_path + waveletType)

    for split in splits:
        if not os.path.exists(main_path + waveletType + '/' + split + '/' + dest_list[0]):
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[0])
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[1])
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[2])

/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest/mexh
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest/morl
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest/gaus1
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest/gaus2


In [5]:
def imageExtraction(full_data,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,channelNum):
    print('Working on Channel: ', channelNum)
    # print('Epochs Data: ',Epochs_data.shape[0])
    
    for waveletType in waveletsTypes:
        ts = time.time()
        i = channelNum
        window = []
        coef,_ = pywt.cwt(full_data[i], scales , waveletType, method = 'conv')
        for j in range(Epochs_data.shape[0]):
            if(not os.path.isfile(main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png')):
                sig_cwt,_ = pywt.cwt(Epochs_data[j][i], scales , waveletType,method = 'conv')
                if Label_data[j][i] == 1:
                    #print(1)
                    plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                    plt.axis('off')
                    plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                    plt.clf()
                    plt.close()

                elif Label_data[j][i] == 2:
                    #print(4)
                    plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                    plt.axis('off')
                    plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[2] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                    plt.clf()
                    plt.close()
                
                else:
                    if j % 30 == 0:
                        plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                        plt.axis('off')
                        plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[0] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                        plt.clf()
                        plt.close()
            collected = gc.collect()    
        te = time.time()
        # print('Tota Time Taken for Channel:',channelNum, te-ts)

In [6]:
def normal_image_extraction(full_data,scales,main_path,split,dest_list,file_num,Epochs_data,channelNum,win_ch):
    i = channelNum
    print("Compiling",i)
    for waveletType in waveletsTypes:
        # for i in range(18):
        coef,_ = pywt.cwt(full_data[i], scales,waveletType,method = 'conv')
        for j in range(win_ch):
            if(not os.path.isfile(main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png')):
                rand_num = randint(0,Epochs_data.shape[0]-1)
                sig_cwt,_ = pywt.cwt(Epochs_data[rand_num][i], scales , 'mexh',method = 'conv')
                plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                plt.axis('off')
                plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[0] + '/' + 'img_' + str(rand_num) + '_' + str(i) + '_' +  str(file_num) + '.png', bbox_inches = 'tight')
                plt.clf()
                plt.close()
            gc.collect()

In [7]:
completed_files = []
if os.path.exists("processed_files.npz"):
    completed_files = np.load("processed_files.npz")['data']

for split in splits:
    if split == 'train':
        split_files_abnormal = train_abnormal
        split_files_normal = train_normal
    elif split == 'test':
        split_files_abnormal = test_abnormal
        split_files_normal = test_normal
    else:
        split_files_abnormal = valid_abnormal
        split_files_normal = valid_normal

    print('Split Type: ', split)
    print('Split Files :', split_files_abnormal)
    print('Split Files :', split_files_normal)
        
    for i,file in enumerate(split_files_abnormal):
        print("Processing:",i+1,"/",len(split_files_abnormal),file)
        if(file not in completed_files):
            file_num = int(file[:-4])
            edf_name = str(10000000 + file_num)[1:] + '.edf' 
            raw = mne.io.read_raw_edf(edfdir + "/" + edf_name,preload = True,exclude = ['A1','A2'],verbose=False)     # Importing all EEG Channels, exculding A1 A2 since matlab has already refrenced the channels with A1 and A2
            raw.filter(l_freq=1,h_freq=45,verbose=False)      # Bandpass filtering [1-45] Hz
            full_data = np.array(raw.get_data())
            epochs=mne.make_fixed_length_epochs(raw,duration=2,overlap=1,verbose=False)           #Setting overlapping duration of 1 second
            epochs_data=epochs.get_data(verbose=False)

            type,channels,beg,end = extractlabels(file_num,csvdir)
            type = cleanlabels(type)
            type, channels, beg, end = np.array(type), np.array(channels), np.array(beg), np.array(end)
            labels = np.array(generatelabelarray(type, channels))
            data = np.array(raw.get_data())
            label_data = np.transpose(sample_labeling(data,labels,beg,end))
            
            label_data = int_encoder(label_data)
            info_labels = mne.create_info(ch_names=['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'FZ', 'PZ', 'CZ'],sfreq = 200)   #Setting channels in MNE Object
            raw_labels = mne.io.RawArray(label_data,info_labels,verbose=False)                #Making MNE Raw object and making overlapping epochs of labels per sample
            epochs_labels=mne.make_fixed_length_epochs(raw_labels,duration=2,overlap=1,verbose=False)         #Setting overlapping duration of 1 second
            epochs_labels = epochs_labels.get_data(verbose=False)
            epochs_labels = epoch_windowing(epochs_labels) #(no of epochs, channels)
            Label_data = np.array(Label_data, dtype = int)

            Label_data = epochs_labels
            Epochs_data = epochs_data
            
            num_processes = Epochs_data.shape[1]

            print("Epochs data shape",Epochs_data.shape)
            # You can provide a list of tuples as the second argument to map, where each tuple contains the input arguments
            if(Epochs_data.shape[0] < 4000):
                inputs = [(full_data,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,i) for i in range(num_processes)]
                with multiprocessing.Pool(processes=num_processes-1, maxtasksperchild=10) as pool:
                    pool.starmap(imageExtraction, inputs)
        
            else:    
                for i in range(num_processes):
                    imageExtraction(full_data,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,i)
                    collected = gc.collect()

            completed_files = np.append(completed_files,file)
            np.savez("processed_files.npz",data=completed_files)

        collected = gc.collect()    
    collected = gc.collect()

    print('---------------------Going now Normal---------------------')
    random.seed(444)     #Make sure the seed is same to get similar results
    window_num = 300     #Define the number of windows you want per normal file
    '''
        This is done to downsample from the large amount of normal data present. This is done to mitigate the data imbalance
        You can get the total number of normal windows(images) produced = window_num * no of files. 
    '''
    win_ch = int(window_num/19)
    coef_data = np.empty((2,19))

    for i,file in enumerate(split_files_normal):
        print("Processing:",i+1,"/",len(split_files_normal),file)
        if(file not in completed_files):
            file_num = int(file[:-4])
            raw = mne.io.read_raw_edf(normalEdf_dir + file,preload = True,exclude = ['A1','A2'],verbose=False)     # Importing all EEG Channels, exculding A1 A2 since matlab has already refrenced the channels with A1 and A2
            raw.filter(l_freq=0.5,h_freq=45,fir_window='hamming',verbose=False)      # Bandpass filtering [1-45] Hz
            full_data = raw.get_data(verbose=False)
            epochs=mne.make_fixed_length_epochs(raw,duration=2,overlap=0,verbose=False)
            epochs_data=epochs.get_data(verbose=False)  
            
            print('Shape of input data after Epochs:',epochs_data.shape)
            num_processes = 18
            inputs = [(full_data,scales,main_path,split,dest_list,file_num,epochs_data,i,win_ch) for i in range(num_processes)]
            
            with multiprocessing.Pool(processes=num_processes-1, maxtasksperchild=10) as pool:
                pool.starmap(normal_image_extraction, inputs)

            
            collected = gc.collect()
            completed_files = np.append(completed_files,file)
            np.savez("processed_files.npz",data=completed_files)


Split Type:  train
Split Files : ['0000502.edf', '0000103.edf', '0000138.edf', '0000400.edf', '0000004.edf', '0000586.edf', '0000391.edf', '0000993.edf', '0000354.edf', '0001086.edf', '0000112.edf', '0000556.edf', '0000790.edf', '0000538.edf', '0000676.edf', '0000893.edf', '0000964.edf', '0000952.edf', '0000247.edf', '0001066.edf', '0000545.edf', '0000700.edf', '0000507.edf', '0000281.edf', '0001020.edf', '0000361.edf', '0000180.edf', '0000510.edf', '0000828.edf', '0000956.edf', '0000827.edf', '0000919.edf', '0000131.edf', '0000624.edf', '0001004.edf', '0000306.edf', '0000594.edf', '0000278.edf', '0000765.edf', '0000652.edf', '0001005.edf', '0000627.edf', '0000353.edf', '0000884.edf', '0001082.edf', '0000623.edf', '0000598.edf', '0000620.edf', '0001065.edf', '0001022.edf', '0001018.edf', '0000753.edf', '0000965.edf', '0000289.edf', '0000638.edf', '0000006.edf', '0000734.edf', '0000637.edf', '0000744.edf', '0000250.edf', '0000635.edf', '0000222.edf', '0000864.edf', '0000437.edf', '00008

/tmp/ipykernel_4778/2772422829.py:84: RuntimeWarning: filter_length (1321) is longer than the signal (738), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=0.5,h_freq=45,fir_window='hamming',verbose=False)      # Bandpass filtering [1-45] Hz


CompilingCompilingCompilingCompiling  Compiling Compiling01  24
 3

Compiling
Compiling
5Compiling 
Compiling 6 CompilingCompiling
 Compiling79 Compiling
Compiling  
 12 1311CompilingCompiling14


 
8 10
1516


Compiling 17
Processing: 100 / 116 0000037.edf
Shape of input data after Epochs: (904, 19, 200)
Compiling 0
Compiling 1
Compiling 2
Compiling 3
Compiling 4
Compiling 5
Compiling 6
Compiling 7
Compiling 8
Compiling 9
Compiling 10
Compiling 11
Compiling 12
Compiling 13
Compiling 14
Compiling 15
Compiling 16
Compiling 17
Processing: 101 / 116 0000084.edf
Shape of input data after Epochs: (866, 19, 200)
Compiling 0
Compiling 1
Compiling 2
Compiling 3
Compiling 4
Compiling 5
Compiling 6
Compiling 7
Compiling 8
Compiling 9
Compiling 10
Compiling 11
Compiling 12
Compiling 13
Compiling 14
Compiling 15
Compiling 16
Compiling 17
Processing: 102 / 116 0000246.edf
Shape of input data after Epochs: (427, 19, 400)
Compiling 0
Compiling 1
Compiling 2
Compiling 3
Compiling 4
Compiling 5
Compili